In [23]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [53]:
def average_combined_point_cloud(folder, pattern):
    folder = Path(folder)
    files = list(folder.rglob(pattern))

    all_clouds = []
    for file in files:
        df = pd.read_csv(file, header=None) # delim_whitespace=True
        all_clouds.append(df.to_numpy())

    # Ensure all point clouds have the same number of points
    lengths = [cloud.shape[0] for cloud in all_clouds]
    if len(set(lengths)) != 1:
        raise ValueError(f"Point clouds have different lengths: {lengths}")
    
    stacked = np.stack(all_clouds)  # Shape: (N_files, N_points, 3)
    avg_cloud = np.mean(stacked, axis=0)  # Shape: (N_points, 3)

    return avg_cloud



In [54]:
# Creating ED average point cloud
folder_path = "Aligned_Models"
avg_cloud = average_combined_point_cloud(folder_path, "*.txt")
np.savetxt("avg_points_cloud_ED.txt", avg_cloud, fmt="%.8f", delimiter=",")


In [ ]:
# # Save ES points seperated into endo and epi in same structure as ED
# input_folder = "ES_pointclouds_merged"
# output_folder = "ES_pointclouds"

# os.makedirs(output_folder, exist_ok=True)

# for filename in os.listdir(input_folder):
#     if filename.startswith("MAD_") and filename.endswith("_merged_points.txt"):
#         # Extract participant ID (number after MAD_)
#         parts = filename.split("_")
#         participant_id = parts[1]

#         filepath = os.path.join(input_folder, filename)
#         df = pd.read_csv(filepath, header=None, names=["x", "y", "z"])

#         # Split into epi and endo
#         df_epi = df.iloc[:801]
#         df_endo = df.iloc[801:1602]

#         # Create participant folder and save
#         participant_folder = os.path.join(output_folder, participant_id)
#         os.makedirs(participant_folder, exist_ok=True)
#         df_epi.to_csv(os.path.join(participant_folder, "points_cloud_epi_ES.csv"), index=False, header=False)
#         df_endo.to_csv(os.path.join(participant_folder, "points_cloud_endo_ES.csv"), index=False, header=False)

#         #print(f"Processed participant {participant_id}")



In [63]:
def compute_and_save_motion(idx, ed_folder, es_folder, output_folder, avg_path):
    ed_file = os.path.join(ed_folder, f"{idx}_merged_points.txt")
    es_file = os.path.join(es_folder, f"MAD_{idx}_merged_points.txt")
    out_file = os.path.join(output_folder, f"{idx}_merged_points.txt")

    if not os.path.exists(ed_file) or not os.path.exists(es_file):
        print(f"Skipping {idx}: missing file(s)")
        return

    ed = pd.read_csv(ed_file, header=None)
    es = pd.read_csv(es_file, header=None)
    average_ed = pd.read_csv(avg_path, header=None)
    #print(f"Shapes: ed={ed.shape}, es={es.shape}, average_ed={average_ed.shape}")

    if ed.shape != es.shape or ed.shape != average_ed.shape:
        
        print(f"Skipping {idx}: shape mismatch")
        return

    motion = ed - es + average_ed

    os.makedirs(output_folder, exist_ok=True)
    motion.to_csv(out_file, sep=',', index=False, header=False)
    print(f"Saved: {idx}_merged_points.txt")

In [65]:
# Paths
ed_folder = "Aligned_Models"
es_folder = "ES_pointclouds_merged"
output_folder = "pc_motion"
avg_path = "avg_points_cloud_ED.txt"

# Loop through all ED files
for fname in os.listdir(ed_folder):
    if fname.endswith("_merged_points.txt"):
        try:
            idx = fname.split("_")[0]  # gets the number before '_merged_points.txt'
            compute_and_save_motion(idx, ed_folder, es_folder, output_folder, avg_path)
        except Exception as e:
            print(f"Failed on {fname}: {e}")

Saved: 170_merged_points.txt
Saved: 188_merged_points.txt
Saved: 158_merged_points.txt
Saved: 144_merged_points.txt
Saved: 12_merged_points.txt
Saved: 95_merged_points.txt
Saved: 175_merged_points.txt
Saved: 141_merged_points.txt
Saved: 17_merged_points.txt
Saved: 127_merged_points.txt
Saved: 169_merged_points.txt
Saved: 171_merged_points.txt
Saved: 91_merged_points.txt
Saved: 27_merged_points.txt
Saved: 145_merged_points.txt
Saved: 75_merged_points.txt
Saved: 123_merged_points.txt
Saved: 94_merged_points.txt
Saved: 112_merged_points.txt
Saved: 190_merged_points.txt
Skipping 70: missing file(s)
Saved: 168_merged_points.txt
Saved: 58_merged_points.txt
Saved: 126_merged_points.txt
Saved: 92_merged_points.txt
Saved: 146_merged_points.txt
Saved: 76_merged_points.txt
Skipping 108: missing file(s)
Saved: 120_merged_points.txt
Saved: 97_merged_points.txt
Saved: 139_merged_points.txt
Saved: 21_merged_points.txt
Saved: 73_merged_points.txt
Saved: 15_merged_points.txt
Saved: 8_merged_points.txt


# How to run without overwriting results
* save pc_motion folder outside of this path and rename to Aligned_Models
* copy the results folder, since saxomode-pca takes participant number from results folder
* then call saxomode-pca
* resulting scores are in PCA_results